In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [4]:
mainPath1 = './templates/contents/contents_1/귀어귀촌개념.html'
mainPath2 = './templates/contents/contents_1/귀어귀촌준비절차.html'
mainPath3 = './templates/contents/contents_1/어업면허허가.html'
mainPath4 = './templates/contents/contents_1/어업정의.html'
mainPath5 = './templates/contents/contents_2/귀어정책지원교육.html'
mainPath6 = './templates/contents/contents_2/양식장및어선임대사업.html'
mainPath7 = './templates/contents/contents_2/어촌이사혜택.html'
mainPath8 = './templates/contents/contents_2/청년어촌정착지원금.html'
mainPath9 = './templates/contents/contents_3/유해해양생물_대응법.html'
mainPath10 = './templates/contents/contents_3/유해해양생물_종류.html'
mainPath11 = './templates/contents/contents_3/유해해양생물.html'
mainPath12 = './templates/contents/contents_3/이상생태현상_굴폐사.html'
mainPath13 = './templates/contents/contents_3/이상생태현상_대응법.html'
mainPath14 = './templates/contents/contents_3/이상생태현상_산호초백화.html'
mainPath15 = './templates/contents/contents_3/이상생태현상.html'
mainPath16 = './templates/contents/contents_4/IoTAI활용스마트양식.html'
mainPath17 = './templates/contents/contents_4/친환경지속가능어업.html'
mainPath18 = './templates/contents/contents_5/어업종류.html'
mainPath19 = './templates/contents/contents_5/지역별주력어종.html'

file_path = [
            mainPath1,
            mainPath2,
            mainPath3,
            mainPath4,
            mainPath5,
            mainPath6,
            mainPath7,
            mainPath8,
            mainPath9,
            mainPath10,
            mainPath11,
            mainPath12,
            mainPath13,
            mainPath14,
            mainPath15,
            mainPath16,
            mainPath17,
            mainPath18,
            mainPath19
             ]
contentList = []

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # 총 길이
    chunk_overlap=50,   # 겹치는 길이
    separators=["\n\n", "\n", ".", " ", ""],  # 끊는 우선순위
)



for i in file_path:
    with open(i, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    folder = os.path.basename(os.path.dirname(i)) 
    page = os.path.splitext(os.path.basename(i))[0]  
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = soup.select('[name="section"]')
    titles = None
    contents = None
    title = None
    content = None
    for j in sections:
        titles = j.select('[name="content_title"]')
        contents = j.select('[name="content_text"]')
        # 둘 다 1개 이상일 때
        if len(titles) > 1 and len(contents) > 1:
            count = min(len(titles), len(contents))  # 길이 맞추기
            for k in range(count):
                title = titles[k].get_text(strip=True)

                content = contents[k].get_text(strip=True)
                

                chunks = splitter.split_text(content)
                for k, chunk in enumerate(chunks):
                    contentList.append([title, chunk, folder, page])

        # 하나만 있을 경우 (정확히 하나일 때만 출력)
        elif len(titles) == 1 and len(contents) == 1:
            title = titles[0].get_text(strip=True)
            content = contents[0].get_text(strip=True)
            chunks = splitter.split_text(content)
            for k, chunk in enumerate(chunks):
                contentList.append([title, chunk, folder, page])
        else:
            print(f"{f}의 제목 또는 내용이 없습니다.")
            # print(j)

df = pd.DataFrame(contentList,columns=["title", "content", "folder", "page"])
df



,title,content,folder,page
0,귀어/귀어업,어업활동을 하기 위해 타지에서부터 어촌으로 이사와 거주하는 것,contents_1,귀어귀촌개념
1,귀촌/귀어촌,귀촌/귀어촌어업활동과 관련없이 타지에서 어촌으로 이주하는 것,contents_1,귀어귀촌개념
2,귀어귀촌 준비절차,"STEP 01.귀어를 준비하는 사람은 수산 관련기관 홈페이지에서 정보를 수집하고, ...",contents_1,귀어귀촌준비절차
3,귀어귀촌 준비절차,".행정 절차를 완료한 후, 수립한 계획에 따라 사업을 본격적으로 추진한다.STEP ...",contents_1,귀어귀촌준비절차
4,어업면허,어업면허는 특정한 수면을 독점적으로 사용할 수 있도록 국가가 권리를 부여하는 제도입...,contents_1,어업면허허가
...,...,...,...,...
249,"기계적 어구류(機械的 漁具類, Harvesting machines)",기계적 어구류,contents_5,어업종류
250,지역별 주력 어종,1. 서울특별시\n 어업 유형: 내수면 어업 중심\n 주요...,contents_5,지역별주력어종
251,지역별 주력 어종,양식 대상 어종: 메기\n 6. 대전광역시\n 어업 유형:...,contents_5,지역별주력어종
252,지역별 주력 어종,"주요 어종: 명태, 오징어, 대구\n 양식 대상 어종: 대구\n ...",contents_5,지역별주력어종


In [5]:
# 2. 문서 객체 생성
documents = [
    Document(page_content=row['content'], metadata={"title": row['title'], "folder": row['folder'], "page": row['page']})
    for _, row in df.iterrows()
]

embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)

vectorstore.save_local("faiss_store/contents")